In [1]:
from benchmark.molecule import DiatomicMolecularHamiltonian

molecule = DiatomicMolecularHamiltonian(6, "sto-3g", "H", "H", 0.7414, True)

Starting to parse FermionOperator using 6 qubits...

Operator t:  (-0.09886396933545855+0j) []
Term, coeff:  () (-0.09886396933545855+0j)
Operator t:  (0.17119774903433+0j) [Z0]
Term, coeff:  ((0, 'Z'),) (0.17119774903433+0j)
Index, p_char:  0 Z
Operator t:  (0.17119774903432997+0j) [Z0 Z1]
Term, coeff:  ((0, 'Z'), (1, 'Z')) (0.17119774903432997+0j)
Index, p_char:  0 Z
Index, p_char:  1 Z
Operator t:  (-0.22278593040418473+0j) [Z2]
Term, coeff:  ((2, 'Z'),) (-0.22278593040418473+0j)
Index, p_char:  2 Z
Operator t:  (-0.2227859304041847+0j) [Z1 Z2 Z3]
Term, coeff:  ((1, 'Z'), (2, 'Z'), (3, 'Z')) (-0.2227859304041847+0j)
Index, p_char:  1 Z
Index, p_char:  2 Z
Index, p_char:  3 Z
Operator t:  (0.1686221915892095+0j) [Z1]
Term, coeff:  ((1, 'Z'),) (0.1686221915892095+0j)
Index, p_char:  1 Z
Operator t:  (0.04532220205287401+0j) [Y0 Z1 Y2 Z3]
Term, coeff:  ((0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z')) (0.04532220205287401+0j)
Index, p_char:  0 Y
Index, p_char:  1 Z
Index, p_char:  2 Y
Index, p_